# 6. FNSIT Toy Example (1D and Tiny 2D)

This notebook is a learning-first, toy example of **Fast Non-Stationary Iterated Tikhonov (FNSIT)**.
We show: (1) the original signal, (2) a blurred measurement, (3) the noisy measurement, and (4) FNSIT reconstruction.

Key idea (NSIT):
$$x_n = x_{n-1} + (A^T A + lpha_n I)^{-1} A^T (y - A x_{n-1})$$

FNSIT uses a **fast approximate inner solve** instead of an exact linear solve.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
plt.rcParams['figure.figsize'] = (12, 4)

ModuleNotFoundError: No module named 'numpy'

## Helper functions (explicit, no black-box NSIT/FNSIT calls)

In [ ]:
def make_time_grid(n):
    return np.linspace(0.0, 1.0, n)


def make_true_signal(t):
    base = 0.6 * np.sin(2 * np.pi * 3 * t)
    bump = np.where((t > 0.35) & (t < 0.55), 1.0, 0.0)
    spike = np.exp(-((t - 0.75) ** 2) / (2 * 0.002))
    return base + 0.6 * bump + 0.9 * spike


def gaussian_kernel_1d(size=9, sigma=2.0):
    assert size % 2 == 1, "size must be odd"
    r = size // 2
    x = np.arange(-r, r + 1)
    k = np.exp(-(x ** 2) / (2 * sigma ** 2))
    k = k / np.sum(k)
    return k


def blur_matrix_1d(n, kernel):
    # Zero-padding blur matrix (Toeplitz-like)
    r = len(kernel) // 2
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            k_idx = j - i + r
            if 0 <= k_idx < len(kernel):
                A[i, j] = kernel[k_idx]
    return A


def add_gaussian_noise(y, noise_std):
    noise = noise_std * np.random.randn(*y.shape)
    return y + noise, noise


def nsit_update_exact(A, r, alpha):
    # Solve (A^T A + alpha I) z = A^T r
    ATA = A.T @ A
    rhs = A.T @ r
    n = A.shape[1]
    z = np.linalg.solve(ATA + alpha * np.eye(n), rhs)
    return z


def nsit_update_fast(A, r, alpha, inner_steps=5, step=0.2):
    # Approximate solve with a few gradient steps
    ATA = A.T @ A
    rhs = A.T @ r
    n = A.shape[1]
    z = np.zeros(n)
    for _ in range(inner_steps):
        grad = (ATA + alpha * np.eye(n)) @ z - rhs
        z = z - step * grad
    return z


def run_fnsit(A, y, x_true, alpha0=1e-1, q=0.7, max_iter=50,
             inner_steps=5, step=0.2, tau=1.05, delta=None,
             use_exact=False):
    x = np.zeros(A.shape[1])
    residuals = []
    errors = []
    for n in range(max_iter):
        alpha_n = alpha0 * (q ** n)
        r = y - A @ x
        if use_exact:
            update = nsit_update_exact(A, r, alpha_n)
        else:
            update = nsit_update_fast(A, r, alpha_n, inner_steps, step)
        x = x + update

        res_norm = np.linalg.norm(A @ x - y)
        residuals.append(res_norm)
        if x_true is not None:
            err = np.linalg.norm(x - x_true) / np.linalg.norm(x_true)
            errors.append(err)

        if delta is not None and res_norm <= tau * delta:
            break

    return x, residuals, errors

## Part A: 1D toy signal (see original, blurred, noisy)

In [ ]:
# Build 1D toy problem
n = 120
t = make_time_grid(n)
x_true = make_true_signal(t)

kernel = gaussian_kernel_1d(size=9, sigma=2.0)
A = blur_matrix_1d(n, kernel)

y_clean = A @ x_true
noise_std = 0.03
y_noisy, noise = add_gaussian_noise(y_clean, noise_std)
delta = np.linalg.norm(noise)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
axes[0].plot(t, x_true, 'k', linewidth=2)
axes[0].set_title('Original signal')
axes[0].grid(True)

axes[1].plot(t, y_clean, 'b', linewidth=2)
axes[1].set_title('Blurred (A x)')
axes[1].grid(True)

axes[2].plot(t, y_noisy, 'r', linewidth=2)
axes[2].set_title('Noisy measurement (A x + noise)')
axes[2].grid(True)

plt.tight_layout()
plt.show()

## Part B: FNSIT reconstruction (fast inner solve)

In [ ]:
# Run exact NSIT and fast NSIT (FNSIT) for comparison
x_exact, res_exact, err_exact = run_fnsit(
    A, y_noisy, x_true, alpha0=1e-1, q=0.7, max_iter=60,
    tau=1.05, delta=delta, use_exact=True
)

x_fast, res_fast, err_fast = run_fnsit(
    A, y_noisy, x_true, alpha0=1e-1, q=0.7, max_iter=60,
    inner_steps=4, step=0.2, tau=1.05, delta=delta, use_exact=False
)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(t, x_true, 'k', linewidth=2, label='True')
axes[0].plot(t, x_exact, 'g--', linewidth=2, label='NSIT (exact)')
axes[0].plot(t, x_fast, 'm-.', linewidth=2, label='FNSIT (fast)')
axes[0].set_title('Reconstruction comparison')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(res_exact, 'g', label='NSIT residual')
axes[1].plot(res_fast, 'm', label='FNSIT residual')
axes[1].axhline(1.05 * delta, color='r', linestyle='--', label='Morozov')
axes[1].set_title('Residuals vs iteration')
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Residual norm')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print('Final relative error (exact):', err_exact[-1])
print('Final relative error (fast): ', err_fast[-1])

## Part C: Tiny 2D image toy (16x16)

In [ ]:
# Create a tiny 2D image with simple shapes
n2 = 16
img = np.zeros((n2, n2))
img[4:7, 4:7] = 1.0
img[10:13, 9:12] = 0.7
img[2:4, 11:14] = 0.9

# Build 2D blur via Kronecker product of 1D blur
k2 = gaussian_kernel_1d(size=5, sigma=1.2)
B = blur_matrix_1d(n2, k2)
A2 = np.kron(B, B)

x2 = img.reshape(-1)
y2_clean = A2 @ x2
noise_std2 = 0.02
y2_noisy, noise2 = add_gaussian_noise(y2_clean, noise_std2)
delta2 = np.linalg.norm(noise2)

# FNSIT on 2D
x2_rec, res2, err2 = run_fnsit(
    A2, y2_noisy, x2, alpha0=1e-1, q=0.7, max_iter=40,
    inner_steps=4, step=0.2, tau=1.05, delta=delta2, use_exact=False
)

img_blur = y2_clean.reshape(n2, n2)
img_noisy = y2_noisy.reshape(n2, n2)
img_rec = x2_rec.reshape(n2, n2)

fig, axes = plt.subplots(1, 4, figsize=(14, 4))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(img_blur, cmap='gray')
axes[1].set_title('Blurred')
axes[1].axis('off')

axes[2].imshow(img_noisy, cmap='gray')
axes[2].set_title('Noisy')
axes[2].axis('off')

axes[3].imshow(img_rec, cmap='gray')
axes[3].set_title('FNSIT reconstruction')
axes[3].axis('off')

plt.tight_layout()
plt.show()

## Notes
- FNSIT here uses a few gradient steps to **approximate** the inner solve.
- You can adjust `inner_steps` and `step` to see the speed vs accuracy trade-off.
- The Morozov threshold uses $\|A x - y\| \le \tau \delta$ with $\tau=1.05$.